In [1]:
#import libraries
import numpy as np
import pandas as pd
import psycopg2

### Data Collection

Connect notebook to local copy of MIMIC-III database and execute postgreSQL queries to store data tables in Pandas dataframes

In [2]:
# below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('ggplot')

# information used to create a database connection
sqluser = 'postgres'
dbname = 'mimic'
schema_name = 'mimiciii'

# Connect to postgres with a copy of the MIMIC-III database
con = psycopg2.connect(dbname=dbname, user=sqluser,password = '*******')

# the below statement is prepended to queries to ensure they select from the right schema
query_schema = 'set search_path to ' + schema_name + ';'

In [3]:
# Create dictionary of ranges for lab data 
# Keys should match the lab names in the query below
# Are these values correct given the units? Need to check.

lab_ranges = {'BICARBONATE': [22,32],
              'BUN': [6,20],
              'CALCIUM': [8.4,10.3],
              'CHLORIDE': [96,108],
              'CREATININE': [0.4,1.1],
              'HEMOGLOBIN': [11.2,15.7],
              'LACTATE': [0.5,2.0],
              'MAGNESIUM': [1.6,2.6],
              'PHOSPHATE': [2.7,4.5],
              'PLATELET': [150,400],
              'POTASSIUM': [3.3,5.1],
              'SODIUM': [133,145]
             }

 ### Execute queries to load in tables from MIMIC-III Database stored on my local computer
 
The cells below execute postGRESQL queries to select and store the columns of the following tables in pandas dataframes, and joins them along common columns.
 
 $\textbf{Admissions Table:}$ subject_id, hadm_id, admittime, dischtime, deathtime 
 
 $\textbf{CPTEvents Table:}$ subject_id, hadm_id, chartdate, cpt_cd, cpt_number, cpt_suffix, description 
 
 $\textbf{Patients Table:}$  all columns

In [4]:
admissions_query = query_schema+ """
SELECT subject_id, hadm_id, admittime, dischtime, deathtime
FROM admissions
"""

cptevents_query = query_schema + """
SELECT subject_id, hadm_id, chartdate, cpt_cd, cpt_number, cpt_suffix, description
FROM cptevents 
"""

patients_query = query_schema + """
SELECT *
FROM patients
"""


#execute queries and store tables in pandas dataframel
admissions_df = pd.read_sql_query(admissions_query, con)
cptevents_df = pd.read_sql_query(cptevents_query, con)
patients_df = pd.read_sql_query(patients_query, con)

In [5]:
# Connect to local postgres version of mimic
con = psycopg2.connect(dbname=dbname, user=sqluser,password = '*******')
cur = con.cursor()
cur.execute('SET search_path to {}'.format(schema_name))

# Modify first day labs code: 
# https://github.com/MIT-LCP/mimic-code/tree/master/etc/firstday
query = \
"""
WITH pvt AS (
  SELECT ie.subject_id, ie.hadm_id, ie.outtime, ie.icustay_id, le.charttime, ad.deathtime
  , CASE when ad.deathtime between ie.intime and ie.outtime THEN 1 ELSE 0 END AS mort_icu
  , CASE when ad.deathtime between ad.admittime and ad.dischtime THEN 1 ELSE 0 END AS mort_hosp
  -- here we assign labels to ITEMIDs
  -- this also fuses together multiple ITEMIDs containing the same data
  , CASE
        when le.itemid = 50868 then 'ANION GAP'
        when le.itemid = 50862 then 'ALBUMIN'
        when le.itemid = 50882 then 'BICARBONATE'
        when le.itemid = 50885 then 'BILIRUBIN'
        when le.itemid = 50912 then 'CREATININE'
        when le.itemid = 50806 then 'CHLORIDE'
        when le.itemid = 50902 then 'CHLORIDE'
        when itemid = 50809 then 'GLUCOSE'
        when itemid = 50931 then 'GLUCOSE'
        when itemid = 50810 then 'HEMATOCRIT'
        when itemid = 51221 then 'HEMATOCRIT'
        when itemid = 50811 then 'HEMOGLOBIN'
        when itemid = 51222 then 'HEMOGLOBIN'
        when itemid = 50813 then 'LACTATE'
        when itemid = 50960 then 'MAGNESIUM'
        when itemid = 50970 then 'PHOSPHATE'
        when itemid = 51265 then 'PLATELET'
        when itemid = 50822 then 'POTASSIUM'
        when itemid = 50971 then 'POTASSIUM'
        when itemid = 51275 then 'PTT'
        when itemid = 51237 then 'INR'
        when itemid = 51274 then 'PT'
        when itemid = 50824 then 'SODIUM'
        when itemid = 50983 then 'SODIUM'
        when itemid = 51006 then 'BUN'
        when itemid = 51300 then 'WBC'
        when itemid = 51301 then 'WBC'
      ELSE null
      END AS label
  , -- add in some sanity checks on the values
    -- the where clause below requires all valuenum to be > 0, 
    -- so these are only upper limit checks
    CASE
      when le.itemid = 50862 and le.valuenum >    10 then null -- g/dL 'ALBUMIN'
      when le.itemid = 50868 and le.valuenum > 10000 then null -- mEq/L 'ANION GAP'
      when le.itemid = 50882 and le.valuenum > 10000 then null -- mEq/L 'BICARBONATE'
      when le.itemid = 50885 and le.valuenum >   150 then null -- mg/dL 'BILIRUBIN'
      when le.itemid = 50806 and le.valuenum > 10000 then null -- mEq/L 'CHLORIDE'
      when le.itemid = 50902 and le.valuenum > 10000 then null -- mEq/L 'CHLORIDE'
      when le.itemid = 50912 and le.valuenum >   150 then null -- mg/dL 'CREATININE'
      when le.itemid = 50809 and le.valuenum > 10000 then null -- mg/dL 'GLUCOSE'
      when le.itemid = 50931 and le.valuenum > 10000 then null -- mg/dL 'GLUCOSE'
      when le.itemid = 50810 and le.valuenum >   100 then null -- % 'HEMATOCRIT'
      when le.itemid = 51221 and le.valuenum >   100 then null -- % 'HEMATOCRIT'
      when le.itemid = 50811 and le.valuenum >    50 then null -- g/dL 'HEMOGLOBIN'
      when le.itemid = 51222 and le.valuenum >    50 then null -- g/dL 'HEMOGLOBIN'
      when le.itemid = 50813 and le.valuenum >    50 then null -- mmol/L 'LACTATE'
      when le.itemid = 50960 and le.valuenum >    60 then null -- mmol/L 'MAGNESIUM'
      when le.itemid = 50970 and le.valuenum >    60 then null -- mg/dL 'PHOSPHATE'
      when le.itemid = 51265 and le.valuenum > 10000 then null -- K/uL 'PLATELET'
      when le.itemid = 50822 and le.valuenum >    30 then null -- mEq/L 'POTASSIUM'
      when le.itemid = 50971 and le.valuenum >    30 then null -- mEq/L 'POTASSIUM'
      when le.itemid = 51275 and le.valuenum >   150 then null -- sec 'PTT'
      when le.itemid = 51237 and le.valuenum >    50 then null -- 'INR'
      when le.itemid = 51274 and le.valuenum >   150 then null -- sec 'PT'
      when le.itemid = 50824 and le.valuenum >   200 then null -- mEq/L == mmol/L 'SODIUM'
      when le.itemid = 50983 and le.valuenum >   200 then null -- mEq/L == mmol/L 'SODIUM'
      when le.itemid = 51006 and le.valuenum >   300 then null -- 'BUN'
      when le.itemid = 51300 and le.valuenum >  1000 then null -- 'WBC'
      when le.itemid = 51301 and le.valuenum >  1000 then null -- 'WBC'
    ELSE le.valuenum
    END AS valuenum
  FROM icustays ie

  LEFT JOIN labevents le
    ON le.subject_id = ie.subject_id 
    AND le.hadm_id = ie.hadm_id
    AND le.charttime between (ie.intime - interval '6' hour) 
    AND (ie.intime + interval '1' day)
    AND le.itemid IN
    (
      -- comment is: LABEL | CATEGORY | FLUID | NUMBER OF ROWS IN LABEVENTS
      50868, -- ANION GAP | CHEMISTRY | BLOOD | 769895
      50862, -- ALBUMIN | CHEMISTRY | BLOOD | 146697
      50882, -- BICARBONATE | CHEMISTRY | BLOOD | 780733
      50885, -- BILIRUBIN, TOTAL | CHEMISTRY | BLOOD | 238277
      50912, -- CREATININE | CHEMISTRY | BLOOD | 797476
      50902, -- CHLORIDE | CHEMISTRY | BLOOD | 795568
      50806, -- CHLORIDE, WHOLE BLOOD | BLOOD GAS | BLOOD | 48187
      50931, -- GLUCOSE | CHEMISTRY | BLOOD | 748981
      50809, -- GLUCOSE | BLOOD GAS | BLOOD | 196734
      51221, -- HEMATOCRIT | HEMATOLOGY | BLOOD | 881846
      50810, -- HEMATOCRIT, CALCULATED | BLOOD GAS | BLOOD | 89715
      51222, -- HEMOGLOBIN | HEMATOLOGY | BLOOD | 752523
      50811, -- HEMOGLOBIN | BLOOD GAS | BLOOD | 89712
      50813, -- LACTATE | BLOOD GAS | BLOOD | 187124
      50960, -- MAGNESIUM | CHEMISTRY | BLOOD | 664191
      50970, -- PHOSPHATE | CHEMISTRY | BLOOD | 590524
      51265, -- PLATELET COUNT | HEMATOLOGY | BLOOD | 778444
      50971, -- POTASSIUM | CHEMISTRY | BLOOD | 845825
      50822, -- POTASSIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 192946
      51275, -- PTT | HEMATOLOGY | BLOOD | 474937
      51237, -- INR(PT) | HEMATOLOGY | BLOOD | 471183
      51274, -- PT | HEMATOLOGY | BLOOD | 469090
      50983, -- SODIUM | CHEMISTRY | BLOOD | 808489
      50824, -- SODIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 71503
      51006, -- UREA NITROGEN | CHEMISTRY | BLOOD | 791925
      51301, -- WHITE BLOOD CELLS | HEMATOLOGY | BLOOD | 753301
      51300  -- WBC COUNT | HEMATOLOGY | BLOOD | 2371
    )
    AND le.valuenum IS NOT null 
    AND le.valuenum > 0 -- lab values cannot be 0 and cannot be negative
    
    LEFT JOIN admissions ad
        ON ie.subject_id = ad.subject_id
    AND ie.hadm_id = ad.hadm_id
    
    -- WHERE ie.subject_id < 10000
    
),
ranked AS (
SELECT pvt.*, DENSE_RANK() OVER (PARTITION BY 
    pvt.subject_id, pvt.hadm_id,pvt.icustay_id,pvt.label ORDER BY pvt.charttime) as drank
FROM pvt
)
SELECT r.subject_id, r.hadm_id, r.icustay_id, r.mort_icu, r.mort_hosp
  , max(case when label = 'ANION GAP' then valuenum else null end) as ANIONGAP_1st
  , max(case when label = 'ALBUMIN' then valuenum else null end) as ALBUMIN_1st
  , max(case when label = 'BICARBONATE' then valuenum else null end) as BICARBONATE_1st
  , max(case when label = 'BILIRUBIN' then valuenum else null end) as BILIRUBIN_1st
  , max(case when label = 'CREATININE' then valuenum else null end) as CREATININE_1st
  , max(case when label = 'CHLORIDE' then valuenum else null end) as CHLORIDE_1st
  , max(case when label = 'GLUCOSE' then valuenum else null end) as GLUCOSE_1st
  , max(case when label = 'HEMATOCRIT' then valuenum else null end) as HEMATOCRIT_1st
  , max(case when label = 'HEMOGLOBIN' then valuenum else null end) as HEMOGLOBIN_1st
  , max(case when label = 'LACTATE' then valuenum else null end) as LACTATE_1st
  , max(case when label = 'MAGNESIUM' then valuenum else null end) as MAGNESIUM_1st
  , max(case when label = 'PHOSPHATE' then valuenum else null end) as PHOSPHATE_1st
  , max(case when label = 'PLATELET' then valuenum else null end) as PLATELET_1st
  , max(case when label = 'POTASSIUM' then valuenum else null end) as POTASSIUM_1st
  , max(case when label = 'PTT' then valuenum else null end) as PTT_1st
  , max(case when label = 'INR' then valuenum else null end) as INR_1st
  , max(case when label = 'PT' then valuenum else null end) as PT_1st
  , max(case when label = 'SODIUM' then valuenum else null end) as SODIUM_1st
  , max(case when label = 'BUN' then valuenum else null end) as BUN_1st
  , max(case when label = 'WBC' then valuenum else null end) as WBC_1st

FROM ranked r
WHERE r.drank = 1
GROUP BY r.subject_id, r.hadm_id, r.icustay_id, r.mort_icu, r.mort_hosp, r.drank
ORDER BY r.subject_id, r.hadm_id, r.icustay_id, r.mort_icu, r.mort_hosp, r.drank;
"""

data = pd.read_sql_query(query,con)

### Data Preprocessing

For the SVD implementations we will replace the nan values with an average and for the NMF implementations we will fill them with 0 (because NMF performs better on sparse matrices)

In [6]:
from sklearn import preprocessing
#We will use our threshold of max number of nan values at 7
lab_data = data.dropna(thresh=6)
#construct a lambda function to fill nan values of each column with that average
lab_data = lab_data.apply(lambda x: x.fillna(x.mean())) 

In [7]:
nmf_lab_data = data.apply(lambda x: x.fillna(0))
#construct lambda function to fill nan values with 0
nmf_lab_data = nmf_lab_data.drop(['subject_id','hadm_id','icustay_id'],axis =1)

In [8]:
#zero center lab data for sdd calculations
sdd_lab_data = lab_data.apply(lambda x: x.fillna(x.mean())) 
sdd_labMatrix = sdd_lab_data.values
sdd_labMatrix = preprocessing.scale(sdd_labMatrix)

In [9]:
dicd_procedures_df = pd.read_csv("C:/Users/rachh/OneDrive/Documents/Senior Thesis/Mimic/D_ICD_PROCEDURES.csv")

In [10]:
dicd_procedures_df.head()

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,264,851,Canthotomy,Canthotomy
1,265,852,Blepharorrhaphy,Blepharorrhaphy
2,266,859,Adjust lid position NEC,Other adjustment of lid position
3,267,861,Lid reconst w skin graft,Reconstruction of eyelid with skin flap or graft
4,268,862,Lid reconst w muc graft,Reconstruction of eyelid with mucous membrane ...


In [11]:
#convert the icd9 code and its keys into a dictionary that can be applied to a dataframe column 
subsection_dict = dicd_procedures_df.set_index('ICD9_CODE')['SHORT_TITLE'].to_dict()

In [12]:
procedures_df = pd.read_csv("C:/Users/rachh/Downloads/procedures.csv")

In [13]:
#apply dictionary
procedures_df['icd9_code'] = procedures_df['icd9_code'].map(subsection_dict)

#### D_CPT

In order to make use of the cpt_events table, it was necessary to convert the cpt_codes to their appropriate values. To do so I could use the d_cpt table in the MIMIC-III database. The codes were provided in sections and subsections, so the section ranges were parsed, converted to integers and then ranges, and stored in a range_array

In [14]:
#execute sql query to read in cpt events dictionary to understand cpt codes 
d_cpt_query = query_schema+ """
SELECT * 
FROM d_cpt
"""

d_cpt_df = pd.read_sql_query(d_cpt_query, con)
d_cpt_df.head()

,row_id,category,sectionrange,sectionheader,subsectionrange,subsectionheader,codesuffix,mincodeinsubsection,maxcodeinsubsection
0,1,1,99201-99499,Evaluation and management,99201-99216,Office/other outpatient services,None,99201,99216
1,2,1,99201-99499,Evaluation and management,99217-99220,Hospital observation services,None,99217,99220
2,3,1,99201-99499,Evaluation and management,99221-99239,Hospital inpatient services,None,99221,99239
3,4,1,99201-99499,Evaluation and management,99241-99255,Consultations,None,99241,99255
4,5,1,99201-99499,Evaluation and management,99261-99263,Follow-up inpatient consultations (deleted codes),None,99261,99263


In [15]:
#loop through values in sectionrange column
#entires are strings, so need to parse the strings with - and ; as delimitors, store the two strings, convert them to integers
#and store those as a range 
import re 
range_array = []

def deleteAlpha(string):
    newStr = ""
    for ch in string:
        print(ch)
        if (ch.isalpha()):
            'hey'
        else:
            newStr = newStr+ ch
    string = newStr
    return string

def convertParseStrToInt(strings,range_array):
    for i in range(0,len(strings),2):
        string1 = strings[i]
        string2 = strings[i+1]
        int1 = int(string1)
        int2 = int(string2)
        range_array.append(range(int1,int2))
        
i=0 
for entry in d_cpt_df['subsectionrange']:
    entry = str(entry)
    strings = re.split('[-;]', entry,flags=re.IGNORECASE)
    if (len(strings) == 2):
        string1 = strings[0]
        string2 = strings[1]
        try:
            int1 = int(string1)
            int2 = int(string2)
            range_array.append(range(int1,int2))
        except:
            string1 = deleteAlpha(string1)
            string2 = deleteAlpha(string2)
            int1 = int(string1.lstrip('0'))
            int2 = int(string2.lstrip('0'))
            range_array.append(range(int1,int2))
            
    if (len(strings) == 1):
        string1 = strings[0]
        try: 
            int1 = int(string1)
            range_array.append(range(int1,int1))
        except:
            string1 = deleteAlpha(string1)
            int1 = int(string1.lstrip('0'))
            range_array.append(range(int1,int1))
    i+=1


0
0
0
1
F
0
0
1
5
F
0
5
0
0
F
0
5
7
5
F
1
0
0
0
F
1
2
2
0
F
2
0
0
0
F
2
0
5
0
F
3
0
0
6
F
3
5
7
3
F
4
0
0
0
F
4
3
0
6
F
5
0
0
5
F
5
1
0
0
F
6
0
0
5
F
6
0
4
5
F
7
0
1
0
F
7
0
2
5
F
0
0
1
6
T
0
2
0
7
T
0
2
5
6
T
0
2
5
9
T


In [16]:
blah = ['1','2','3','4']
rando_array = []
def convertParseStrToInt(strings,range_array):
    for i in range(0,len(strings),2):
        string1 = strings[i]
        string2 = strings[i+1]
        int1 = int(string1)
        int2 = int(string2)
        range_array.append(range(int1,int2))

convertParseStrToInt(blah,rando_array)

In [17]:
d_cpt_df['ranges'] = range_array

In [18]:
#create dictionaries to apply to cpt_events table
subsection_dict = d_cpt_df.set_index('ranges')['subsectionheader'].to_dict()

In [19]:
#create subclass so that numbers within range keys can be converted to the correct output
class RangeDict(dict):
    def __getitem__(self, item):
        if type(item) != range: # or xrange in Python 2
            for key in self:
                if item in key:
                    return self[key]
        else:
            return super().__getitem__(item)

In [20]:
def checkkey(this_key,thisdict):
    for key in thisdict:
        if this_key in key:
            #print(thisdict[key])
            break

In [21]:
range_dict = RangeDict(subsection_dict)

In [22]:
#apply dictionary to cptevents_df
decoded_array = []
for i in range(len(cptevents_df)):
    current_value = cptevents_df.cpt_cd[i]
    try:
        current_value = int(current_value)
        decode = range_dict.__getitem__(current_value)
        decoded_array.append(decode)
    except: 
        current_value = deleteAlpha(current_value)
        current_value = int(current_value.lstrip('0'))
        decode = range_dict.__getitem__(current_value)
        decoded_array.append(decode)

G
0
2
7
2
0
0
5
0
T
S
2
9
0
0
G
0
3
6
4
S
2
0
8
3
G
0
1
2
1
G
0
3
6
4
S
2
0
8
3
0
0
5
0
T
G
0
3
6
4
0
2
5
8
T
0
2
5
7
T
0
2
5
7
T
0
2
5
8
T
0
2
5
6
T
G
0
3
6
4
0
2
5
7
T
0
2
5
6
T
0
2
5
7
T
G
0
3
6
4
0
2
5
6
T
G
0
2
7
2
0
2
5
7
T
G
0
3
6
4
S
2
9
0
0
G
0
2
7
2
G
0
2
7
2
G
0
2
7
2
0
2
5
6
T
0
2
5
8
T
0
2
5
6
T
0
2
5
6
T
0
0
7
8
T
0
0
7
9
T
0
0
8
0
T
0
0
8
1
T
0
2
5
6
T
G
0
3
6
4
G
0
0
0
2
0
2
5
6
T


In [23]:
#create procedure column in cptevents_df containing the values for the cpt_id codes 
cptevents_df['procedure'] = decoded_array

In [24]:
#join all of the dataframes together. I'm not sure whether or not I want to include procedures in the entire yet 
master_df = pd.merge(cptevents_df, admissions_df, on = 'subject_id')
master_df = pd.merge(master_df, patients_df, on = 'subject_id')
master_df = pd.merge(master_df, lab_data, on = 'subject_id')
master_df.head()

,subject_id,hadm_id_x,chartdate,cpt_cd,cpt_number,cpt_suffix,description,procedure,hadm_id_y,admittime,...,magnesium_1st,phosphate_1st,platelet_1st,potassium_1st,ptt_1st,inr_1st,pt_1st,sodium_1st,bun_1st,wbc_1st
0,11743,129545,NaT,99232,99232.0,None,None,Hospital inpatient services,129545,2147-02-24 00:13:00,...,1.4,3.7,376.0,4.0,32.9,1.4,14.3,139.0,18.0,9.9
1,11743,129545,NaT,99232,99232.0,None,None,Hospital inpatient services,129545,2147-02-24 00:13:00,...,1.4,3.7,376.0,4.0,32.9,1.4,14.3,139.0,18.0,9.9
2,11743,129545,NaT,99232,99232.0,None,None,Hospital inpatient services,129545,2147-02-24 00:13:00,...,1.4,3.7,376.0,4.0,32.9,1.4,14.3,139.0,18.0,9.9
3,11743,129545,NaT,99232,99232.0,None,None,Hospital inpatient services,129545,2147-02-24 00:13:00,...,1.4,3.7,376.0,4.0,32.9,1.4,14.3,139.0,18.0,9.9
4,11743,129545,NaT,99223,99223.0,None,None,Hospital inpatient services,129545,2147-02-24 00:13:00,...,1.4,3.7,376.0,4.0,32.9,1.4,14.3,139.0,18.0,9.9


In [25]:
#we will also delete the identifier columns from the lab data to have a solely numeric dataframe
lab_data = lab_data.drop(['subject_id','hadm_id','icustay_id'],axis=1)

### Converting to Dummy Variables

In order to apply matrix decompositions to our dataframe we must convert the categorical variables into dummy variables

In [26]:
master_df=master_df.drop(['cpt_cd','cpt_suffix','cpt_number','cpt_suffix'],axis = 1)
master_df = pd.get_dummies(master_df, prefix=['description','procedure','gender'])

In [27]:
import time
from datetime import datetime as dt

In [28]:
#apply a lambda function to all columns w/ datetime type 
master_df['chartdate'] = master_df['chartdate'].apply(lambda x: (x-dt(1970,1,1)).total_seconds())
master_df['admittime'] = master_df['admittime'].apply(lambda x: (x-dt(1970,1,1)).total_seconds())
master_df['dischtime'] = master_df['dischtime'].apply(lambda x: (x-dt(1970,1,1)).total_seconds())
master_df['deathtime'] = master_df['deathtime'].apply(lambda x: (x-dt(1970,1,1)).total_seconds())
master_df['dob'] = master_df['dob'].apply(lambda x: (x-dt(1970,1,1)).total_seconds())
master_df['dod'] = master_df['dod'].apply(lambda x: (x-dt(1970,1,1)).total_seconds())
master_df['dod_hosp'] = master_df['dod_hosp'].apply(lambda x: (x-dt(1970,1,1)).total_seconds())
master_df['dod_ssn'] = master_df['dod_ssn'].apply(lambda x: (x-dt(1970,1,1)).total_seconds())

In [29]:
#for nmf calculations fill nan values with 0, for svd/sdd fill them with the average
master_df = master_df.apply(lambda x: x.fillna(x.mean())) 
nmf_master_df=master_df.fillna(value=0)

I need to drop my ID columns because these are numeric values that are simply identifiers - they have no actual numeric meaning.

I am going to remove the variables chartdate, deathtime, row_id, dod, dod_hosp, dod_ssn, expire_flag, icustay_id, mort_icu and all procedures except for ? maternity care and delivery to perform an analysis on the remaining variables 

In [30]:
cols = ['admittime', 'dischtime','dob','mort_hosp', 'aniongap_1st', 'albumin_1st',
       'bicarbonate_1st', 'bilirubin_1st', 'creatinine_1st', 'chloride_1st',
       'glucose_1st', 'hematocrit_1st', 'hemoglobin_1st', 'lactate_1st',
       'magnesium_1st', 'phosphate_1st', 'platelet_1st', 'potassium_1st',
       'ptt_1st', 'inr_1st', 'pt_1st', 'sodium_1st', 'bun_1st', 'wbc_1st','procedure_Maternity care and delivery']
maternity_df = master_df[cols]

In [31]:
nmf_maternity_df = nmf_master_df[cols]

In [32]:
maternity_matrix = maternity_df.values
nmf_maternity_matrix = nmf_maternity_df.values

In [33]:
nmf_master_matrix = nmf_master_df.values

Standardize lab data 

In [34]:
standardizedLab_df = lab_data.drop(['mort_icu','mort_hosp'],axis=1)
standardizedLab_matrix = standardizedLab_df.values

In [35]:
standardizedLab_matrix = preprocessing.scale(standardizedLab_matrix)

In [36]:
master_df.to_csv("C:/Users/rachh/OneDrive/Documents/Senior Thesis/master_df.csv")
lab_data.to_csv("C:/Users/rachh/OneDrive/Documents/Senior Thesis/lab_data.csv")
standardizedLab_df.to_csv("C:/Users/rachh/OneDrive/Documents/Senior Thesis/standardizedLab_df.csv")
maternity_df.to_csv("C:/Users/rachh/OneDrive/Documents/Senior Thesis/maternity_df.csv")
nmf_master_df.to_csv("C:/Users/rachh/OneDrive/Documents/Senior Thesis/nmf_master_df.to_csv")